# OCEA272 Problem Set 6

In [1]:
# imports
import numpy as np
from importlib import reload

from scipy import optimize

import unyt

from oceanpy import constants
from oceanpy.forces import coriolis

from os_classes.ocea257 import chapter1, chapter2
from os_classes import plotting as os_plotting

from bokeh import plotting
from bokeh import models
from bokeh import layouts

from IPython.display import Image
from IPython.core.display import HTML

# PS6-1 <font color='blue'>(C-R 11-4 and 11-5)</font>  Diamond Head

## 11-5

### Wind blowing at 10 m/s encounters Diamond Head, leading to vertical displacements $\xi$.

In [2]:
# Setup
U = 10 * unyt.m / unyt.s  # Flow speed
hb = 232 * unyt.m  # Height of the volcano
L = 20 * unyt.km   # Width of the volcano
N = 0.02 / unyt.s

### In lecture, we considered a very similar problem, deriving:

### $\xi = - \frac{F_r^2 h_b}{1-F_r^2}$

### with $F_r$ the Froude number

### $F_r \equiv \frac{U}{NH}$.

### Consider first:

### $\frac{U}{N h_b}$

In [3]:
U / (N * hb)

unyt_quantity(2.15517241, '(dimensionless)')

### If the characteristic height of the flow $H$ is much larger than the volcano $H \gg h_b$, then we will have $F_r \ll 1$.  This is, of course, a requirement for stratification.

### And our equation for the vertical displacements reduces to:

### $\xi \approx -F_r^2 h_b$

### i.e. the vertical displacements are much less than the height of the volcano.

### Is the Coriolis (i.e. rotation) important?

### Let us calculate the Rossby number:

### $R_0 = \frac{U}{\Omega L}$

### Hawaii is at $\approx 20^\circ$N

In [4]:
Omega = coriolis.coriolis_parameter(20*unyt.deg)
Omega.to('1/s')

unyt_quantity(4.98810626e-05, '1/s')

In [5]:
R_0 = U / Omega / L
R_0

unyt_quantity(10.0238442, '(dimensionless)')

### With $R_0 \gg 1$, rotational effects are minimal.

## 11-5

### Now we increase $h_b$ to 1000m and $L = 500$km.

In [6]:
hb_2 = 1000 * unyt.m
L_2 = 500*unyt.km

### Let us calculate our dimensionaless quantities:

In [7]:
R_0_2 = U / Omega / L_2
R_0_2

unyt_quantity(0.40095377, '(dimensionless)')

### We see that rotation is now important.

### And for this much larger $h_b$, it is possible that $H \sim h_b$ and we estimate the Froude number:

In [8]:
F_r_2 = U / N / hb_2
F_r_2

unyt_quantity(0.5, '(dimensionless)')

### We may compare $F_r^2$ to $R_0$:

In [9]:
F_r_2**2 / R_0_2

unyt_quantity(0.62351328, '(dimensionless)')

### We appear to be in the regime where both stratification and rotation are important.

----

# PS6-2 <font color='blue'>(C-R 13-2)</font>  Internal waves

## The lunar tide $M2$ with period $P=12.42$hrs drives internal waves along Norway's coast.

## This occurs in a stratified fluid with frequency $N = 2 \times 10^{-3} \, \rm s^{-1}$

In [10]:
# Setup
P = 12.42 * unyt.hour
N = 2e-3 / unyt.s

## Let's consider the oscillatory period

## $P_N = \frac{2\pi}{N}$

In [11]:
P_N = 2 * np.pi / N
P_N.to('hour')

unyt_quantity(0.87266463, 'hr')

## This is much shorter than the tidal period.  As such, we expect to be in the Radiating Wave regime.

## For the tidal oscillation, it is reasonable to take the frequency of our internal waves to be:

## $\omega = \frac{2 \pi}{P}$

In [12]:
omega = 2*np.pi/P
omega.to('1/s')

unyt_quantity(0.00014053, '1/s')

## We have derived for internal waves the following relation:

## $\omega = N \cos\theta$

## giving

## $\cos\theta = \frac{\omega}{N}$

In [13]:
# Numerically
costheta = omega / N
costheta

unyt_quantity(0.07026285, '(dimensionless)')

In [14]:
theta = np.arccos(costheta)
theta # radians

array(1.50047553)

In [15]:
theta_deg = theta * 180. *unyt.deg / np.pi
theta_deg

unyt_quantity(85.97091528, 'degree')

## And negative solutions are also acceptible, i.e. the waves can travel up and down.

## We have:

## $k_x = k \cos\theta$, $k_z = k \sin\theta$

## Now, the energy follows the group velocity $c_g$

## In lecture and the text, we derived for a wind moving over a sinusoidal surface:

## $c_{gx} = U \cos^2 \theta$

## $c_{gz} = U \sin\theta \cos\theta$

## Here we take $U$ as a characteristic velocity of the tides (not given)

In [17]:
# Numerically
cgx = np.cos(theta)**2
cgz = np.sin(theta)*np.cos(theta)
cgx, cgz

(0.004936868418210029, 0.07008919851468776)

## And the angle of propogation relative to the horizontal is provided by the tangent:

## $\beta = \tan^{-1} \frac{c_{gz}}{c_{gx}}$

In [19]:
beta = np.arctan(cgx/cgz) * unyt.rad

In [20]:
beta.to('deg')

unyt_quantity(4.02908472, 'degree')

## So the energy travels primarily horizontally, but has a small vertical component

----

# PS6-3 <font color='blue'>(C-R 13-4)</font>  Lee waves

## We have a wind blowing over a sinusoidal topography with quantities:

In [37]:
U = 10 * unyt.m/unyt.s # wind speed
N = 0.03 / unyt.s      # Stratification frequency
wave = 25 * unyt.km    # Topography wavelength
H = 500 * unyt.m / 2   # Topography amplitude

## Define $k_x = 2 \pi / \lambda$

In [32]:
k_x = 2 * np.pi / wave
k_x

unyt_quantity(0.25132741, '1/km')

In [33]:
k_x * U, N

(unyt_quantity(0.00251327, '1/s'), unyt_quantity(0.03, '1/s'))

## We see that $k_x U < N$ and we are in the long wavelength regime.

## We have solutions of the form:

## $k_z = \pm \sqrt{N^2/U^2 - k_x^2}$ and may ignore the downward solution.

In [34]:
k_z = np.sqrt(N**2/U**2 - k_x**2)
k_z

unyt_quantity(0.00298945, '1/m')

## And we derive a vertical wavelength, $\lambda_z = 2\pi/k_z$

In [36]:
wave_z = 2*np.pi/k_z
wave_z.to('km')

unyt_quantity(2.10178365, 'km')

## We have $\cos\theta = k_x U / N$ which gives:

In [35]:
theta = np.arccos(k_x*U/N) * unyt.rad
theta.to('deg')

unyt_quantity(85.19436748, 'degree')

## Therefore our Lee waves are nearly vertical!

## From lecture (and the text), the horizontal velocity for Lee waves is:

## $u = U + k_x U H \sin(k_x x + k_z z - \omega t)$

## or at the bottom ($z=0$):

## $u = U + k_x U H \sin(k_x x - \omega t)$


## This is maximal when $\sin = 1$

In [39]:
u = U + k_x * U * H
u.to('m/s')

unyt_quantity(10.62831853, 'm/s')

## This occurs when $k_x x - \omega t = \pi/2$

## Last, our topology is:

## $z = H \cos(k_x x - \omega t)$

## And we have $z=0$ at this maximal velocity, i.e. neither a peak nor trough but the point of maximal slope.

----

# PS6-4 <font color='blue'>(C-R 15-1)</font>  Thermal wind

## Consider a region with a thermal gradient $dT/dy = -1^\circ$C/35km.

## Assume $\phi = 40^\circ$N and $T=290$K.

## Our thermal wind equations (derived in lecture) are:

## $\frac{\partial v}{\partial z} = \frac{g \alpha}{f} \frac{\partial T}{\partial x}$

## $\frac{\partial u}{\partial z} = -\frac{g \alpha}{f} \frac{\partial T}{\partial y}$

## with $\alpha = 1/T$

In [48]:
# Setup
T = 290 * unyt.K
g = 9.8 * unyt.m / unyt.s**2
dT_dy = -1 * unyt.K / (35*unyt.km)
# Coriolis
phi = 40*unyt.deg
f = coriolis.coriolis_parameter(phi)

In [49]:
f.to('1/s')

unyt_quantity(9.37457329e-05, '1/s')

## Derive $\partial u/\partial z$

In [50]:
du_dz = -g * (1/T) /  f * dT_dy
du_dz

unyt_quantity(0.01029932, '1/s')

## We now calculate the change in velocity $\Delta u$ at $z = 2$km:

In [52]:
Delta_u = du_dz * 2*unyt.km
Delta_u.to('m/s')

unyt_quantity(20.59863871, 'm/s')

## If we have $u=0$ at $z=0$, then we finish with a wind speed of $u=20$m/s at $z=2$km.

## We have a positive sign, so the direction is East.

----

# PS6-5 <font color='blue'>(C-R 15-4)</font>  Straight of Gibraltar

----

# PS6-5 <font color='blue'>(C-R 15-4)</font>  Gibraltar

## In the narrowest point of the Strait of Gibraltar (Tarifa Narrows), there is a stratified, two-layer fluid with inflowing Atlantic waters on top and outflowing salty Meditteranean waters below.

## The density difference is $\Delta\rho/\rho = 0.002$, the straight is 11km wide and 650m deep.

## We are told that the two layers form an interface with depth 175m at Spain (North) and 225m in Africa (South).

## And we are told the volumetric transport $V$ is equal between the 2 layers.

## This latter condition implies:

## $V_1 = u_1 \rho_1 H_1 = -V_2 = -u_2 \rho_2 H_2$

## With that interface,  $<H_1> = 200$m and $<H_2> = 450$m.

## Therefore $u_1/u_2 \approx -9/4$, i.e. we know the relative speed.

## We may introduce the thermal wind equations to establish another equation for the relative speeds.

## $\frac{\partial u}{\partial z} = \frac{g}{\rho_0 f} \frac{\partial \rho}{\partial y}$

## We may massage this to read:

## $u_1 = u_2 + \frac{g}{\rho_0 f} \frac{\Delta z}{\Delta y} \Delta\rho$

In [28]:
# Some numerics
L = 11 * unyt.km
Dz_Dy = - 50*unyt.m / L
Dp_p = -0.002
g = 9.8 * unyt.m/unyt.s**2
f = 8.5e-5 / unyt.s

## Substitute $u_2 = -u_1 (4/9)$

## $u_1 (1+4/9) = \frac{g}{f} \frac{\Delta z}{\Delta y} \frac{\Delta\rho}{\rho_0}$

In [26]:
u_1 = (g/f) * (Dz_Dy) * Dp_p / (1+4/9.)
u_1

unyt_quantity(0.72562731, 'm/s')

In [27]:
u_2 = -4*u_1/9
u_2

unyt_quantity(-0.32250103, 'm/s')

## Volumetric transport

## $V_1 = u_1  L  H_1$

In [32]:
H_1 = 200 * unyt.m
V_1 = u_1 * L * H_1
V_1.to('km**3/s')

unyt_quantity(0.00159638, 'km**3/s')